# Introduction

# Framework

prepare the data, then ...
define, compile, fit (mae covered in intro to ml) and evaluate, predict
 
how can we tell how good our NN is? one way is to take the input, pass it to the neural netwoork, and then see how close the value that the network returns is to the true value.  the close it is, the better the network did.

we can formalize this idea with a loss function.

there are many different loss functions.

# Code

### Step 0: Prepare the data

- deal with missing values
- one-hot encode categorical features (in exercise: why not label encoding?)
- normalize the input

In [ ]:
# will be entirely hidden (data prep)
# need to mention removing missing values and also ohe-hot encoding categorical features
# another example: https://www.tensorflow.org/tutorials/keras/regression
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
melbourne_file_path = '../input/melbourne-housing-snapshot/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 

# Drop homes with missing values
melbourne_data = melbourne_data.dropna(axis=0)

melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = melbourne_data[melbourne_features]
y = melbourne_data.Price

### Step 1: Define the model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Initialize a neural network
model = keras.Sequential(
    [
        layers.Dense(32, activation='relu', input_shape=(7,)),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ]
)

### Step 2: Compile the model

In [ ]:
model.compile(loss='mae')

### Step 3: Train and evaluate the model

In [ ]:
history = model.fit(X, y, epochs=500, verbose=0, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()

In [ ]:
plot_loss(history)

### Step 4: Generate predictions

In [ ]:
# change to use test data
predictions = model.predict(X)

In [ ]:
print(list(predictions[:5].flatten()))
print(list(y.head()))